In [1]:
from dataloader_prev_preload import WaveformDatasetPreload
from sampler_prev import RandomConsecutiveSampler
from losses import ConsecutiveDifferenceHigherOrderLossBatch, ConsecutiveDifferenceHigherOrderLoss
from swissarmy_prev import SeqModel


import numpy as np
import torch
from tqdm import tqdm 
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

In [2]:
def generate_sine_tensor(num_bits, length):
    # Create an array of integers from 0 to length - 1
    t = np.arange(length)
    # Generate the sine waves for each bit
    sine_tensor = np.zeros((length, num_bits))  # Initialize the tensor
    
    for i in range(num_bits):
        frequency = (np.pi / (2 ** i))  # Calculate frequency based on the number of bits
        sine_tensor[:, i] = np.cos(frequency * (t))  # Fill the tensor with sine values

    return sine_tensor

In [3]:
num_bits = 21
max_len = 100_000
seq_bits = 4
seq_max_len = 2
pred_prev = 3
directory = "digits_two/"
terminal_pad = 11
seq_vocab_len = 10

# Sampler setup as before
batch_size = 2
consecutive_size = 10

In [4]:
t_input = torch.tensor(generate_sine_tensor(num_bits,max_len)).float()
print(t_input.shape)

seq_t = torch.tensor(generate_sine_tensor(seq_bits,seq_max_len)).float()
print(seq_t.shape)

torch.Size([100000, 21])
torch.Size([2, 4])


In [5]:
seq_t

tensor([[ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00],
        [-1.0000e+00,  6.1232e-17,  7.0711e-01,  9.2388e-01]])

In [6]:
dataset = WaveformDatasetPreload(directory, t_input, max_len, terminal_pad, seq_vocab_len, seq_max_len, seq_t,pred_prev)

 


sampler = RandomConsecutiveSampler(dataset, batch_size, consecutive_size)

# DataLoader
dataloader = DataLoader(dataset, batch_sampler=sampler)

In [7]:
print(dataset.wav_data_list[0].shape)
print(dataset.pred_data_list[0].shape)

torch.Size([1, 27424])
torch.Size([1, 27424, 3])


In [8]:
iteration = 0

for batch in dataloader:
    wav_data, t_step, target, file_idx, seq_inputs, prev_target = batch #right now this wraps arround, just fyi.  not sure its a bad thing.

    print("Waveform data:", wav_data.shape)
    #print("Time step:", t_step.shape)
    #print("Target tensor:", target.shape)
    #print("File index:", file_idx.shape)
    #print("File index:", seq_inputs.shape)
    print("Previous Predictions:", prev_target.shape)
    #print(wav_data)
    #print(prev_target)
    iteration = iteration + 1
    if iteration > 10:
        break
    

Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])
Waveform data: torch.Size([20, 1])
Previous Predictions: torch.Size([20, 3])


In [9]:

config = {
    't_seq_bits': seq_bits,  # Example value for the input bit size
    't_seq_len': seq_max_len,    # Example value for the sequence length
    't_bits': num_bits,      # Example value for the bits used in the decoder

    'encoder': {
        't_layer_dim': 0,               # Example hidden layer dimension for encoder
        't_num_layers': 0,                # Example number of layers in the encoder's initial layer
        'fc_layers': 4,                   # Example number of fully connected layers in the encoder
        'encoder_layers': 2,              # Example number of encoder layers
        'one_hot_vocab_len': 10,          # Vocabulary size for one-hot encoding
        'one_hot_embedding_dim': 20       # Embedding dimension for one-hot encoding
    },

    'decoder': {
        't_layer_dim': 0,                # Example hidden layer dimension for decoder
        't_num_layers': 0,                # Example number of layers in the decoder's initial layer
        'fc_layers': 4,                   # Example number of fully connected layers in the decoder
        'decoder_layers': 3,                # Example number of decoder layers
        'pred_prev_dim': pred_prev
    },

    'output': {
        'mse_output_layers': 2,           # Number of layers in the MSE output head
        'mse_dim': 64,                     # Hidden dimension for the MSE output head
        'bce_output_layers': 2,            # Number of layers in the BCE output head
        'bce_dim': 64                      # Hidden dimension for the BCE output head
    }
}


In [10]:
model = SeqModel(config)

In [11]:
model

SeqModel(
  (encoder): SeqEncoder(
    (initial_layer): SwissArmyLayer(
      (t_layers): ModuleList()
      (embedding): Embedding(11, 20, padding_idx=10)
      (layers): ModuleList(
        (0-3): 4 x Linear(in_features=24, out_features=24, bias=True)
      )
    )
    (encoder_layers): ModuleList(
      (0): SwissArmyLayer(
        (t_layers): ModuleList()
        (embedding): Embedding(11, 20, padding_idx=10)
        (layers): ModuleList(
          (0-3): 4 x Linear(in_features=48, out_features=48, bias=True)
        )
      )
      (1): SwissArmyLayer(
        (t_layers): ModuleList()
        (embedding): Embedding(11, 20, padding_idx=10)
        (layers): ModuleList(
          (0-3): 4 x Linear(in_features=72, out_features=72, bias=True)
        )
      )
    )
  )
  (decoder): SeqDecoder(
    (decoder_layers): ModuleList(
      (0): SwissArmyLayer(
        (t_layers): ModuleList()
        (layers): ModuleList(
          (0-3): 4 x Linear(in_features=96, out_features=96, bias=Tru

In [12]:
bce_output, mse_output = model(seq_inputs,file_idx,t_step,prev_target)
print(bce_output.shape, mse_output.shape)

torch.Size([20, 1]) torch.Size([20, 1])


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
prev_target_list = []
mse_loss_fn = nn.MSELoss()
bce_loss_fn = nn.BCELoss()
cdifb_loss = ConsecutiveDifferenceHigherOrderLossBatch(consecutive_size,order=3)
cdif_loss = ConsecutiveDifferenceHigherOrderLoss(consecutive_size,order=3)

num_epochs = 1

# Training loop
for epoch in range(num_epochs):
    model.train()
    
    #for batch in dataloader:
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        wav_data, t_step, target, file_idx, seq_inputs, prev_target = batch
        
        bce_output, mse_output = model(seq_inputs,file_idx,t_step,prev_target)
        # Compute losses
        mse_loss = mse_loss_fn(mse_output*target, wav_data)  # Assuming the target is for MSE # is this accomplishing what i want?
        bce_loss = bce_loss_fn(bce_output, target)  # Assuming the target is for BCE
        cdif = cdif_loss(mse_output*target, wav_data)
        #bc = bc_loss(outputs, targets)
        cdif_b = cdifb_loss(mse_output*target, wav_data)
        
        
        # Combine losses (you can weight them if needed)
        total_loss = mse_loss + 0.2*bce_loss + 1.9*cdif  + 0.5*cdif_b
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
    # Print progress for each epoch
    print(f"Epoch {epoch+1}/{num_epochs} MSE: {mse_loss.item():.6f} BCE: {bce_loss.item():.6f} CDIF: {cdif.item():.6f}  Total Loss: {total_loss.item():.8f}")
    torch.save(model, "new_model.pth")
print("all done sweetheart <3")

Epoch 1/1:   0%|                         | 875/562212 [00:12<2:19:51, 66.89it/s]

In [ ]:
prev_target_list[-1].shape